# Task to create flagged records (silver1)

In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")
bronze_tbl = f"{catalog}.{schema}.gha_ss_taxi_raw_records_bronze"
silver_tbl_1 = f"{catalog}.{schema}.gha_ss_flagged_rides_silver1"

In [0]:
# location to store checkpoint files
# spark.sql(f"create volume if not exists {catalog}.{schema}.checkpoint_files")
checkpoint_path = f"/Volumes/{catalog}/{schema}/checkpoint_files/top_n/gha_ss_flagged_rides_silver1_checkpoint"

In [0]:
from pyspark.sql.functions import *

streamdf = (spark.readStream.table(bronze_tbl)
 .filter(((col("pickup_zip") == col("dropoff_zip")) & (col("fare_amount") > 50)) |
         ((col("trip_distance") < 5) & (col("fare_amount") > 50))
         )
 .select(date_trunc("week", col("tpep_pickup_datetime")).alias("week"),
          col("pickup_zip").alias("zip"),
          col("fare_amount"),
          col("trip_distance"))
 .writeStream
 .outputMode("append")
 .option("checkpointLocation", checkpoint_path)
 .trigger(availableNow=True)
 .toTable(silver_tbl_1)
)

streamdf.awaitTermination()